In [5]:
import os
path = r'C:\Users\dylan\Desktop\cod\cod\cif'
list_of_files = []
for root, dirs, files in os.walk(path):
    for file in files:
        list_of_files.append(os.path.join(root,file))

In [6]:
print(len(list_of_files))
import warnings
from pymatgen.core import Structure
warnings.filterwarnings("ignore")

149963


In [7]:
print(list_of_files[0])

C:\Users\dylan\Desktop\cod\cod\cif\1\00\00\1000000.cif


In [8]:
import pymatgen
from pymatgen.core import Structure, Element

structures = []
composition = []
materialID = []
index = 50000
offset = index
default = 1
stability = []
end = False
for file in list_of_files:
    if(offset > 0):
        offset -= 1
    else:
        if("cif" in file and end == False):
            try:
                structure = Structure.from_file(file)
                mystring = (str(structure))
                for item in mystring.split("\n"):
                    if "Reduced" in item:
                        composition.append(item.strip()[len("Reduced Formula: "):])
                structures.append(structure)
                stability.append(default)
                materialID.append("ENUMERATION" + str(index))
                index += 1
                if(index % 500 == 0):
                    print("Progress: %", index/len(list_of_files) * 100)
                if(index % 60000 == 0):
                    end = True
                    break
            except:
                pass

Progress: % 33.67497316004614
Progress: % 34.00838873588818
Progress: % 34.34180431173023
Progress: % 34.67521988757227
Progress: % 35.008635463414315
Progress: % 35.34205103925635
Progress: % 35.67546661509839
Progress: % 36.00888219094043
Progress: % 36.34229776678247
Progress: % 36.67571334262452
Progress: % 37.00912891846656
Progress: % 37.34254449430859
Progress: % 37.67596007015064
Progress: % 38.00937564599268
Progress: % 38.34279122183472
Progress: % 38.67620679767676
Progress: % 39.0096223735188
Progress: % 39.34303794936084
Progress: % 39.67645352520288
Progress: % 40.00986910104493


In [9]:
import pandas as pd
df = pd.DataFrame()
df['material_id'] = materialID
df['final_structure'] = structures

In [10]:
from ase.db import connect
import pandas as pd
import os
from ase.db import connect
from pymatgen.core import Composition, Structure
from pymatgen.io.ase import AseAtomsAdaptor
from tqdm.autonotebook import tqdm
from scipy.sparse import identity
from scipy.sparse.linalg import eigs
import numpy as np
from aviary.cgcnn.utils import get_cgcnn_input
from aviary.wren.utils import count_wyks, get_aflow_label_spglib
from tqdm.notebook import tqdm
tqdm.pandas()

In [11]:
df['composition'] = df['final_structure'].apply(lambda x: x.composition.reduced_formula)
def newFunc(inputVal):
    try:
        return get_aflow_label_spglib(inputVal)
    except:
        return "NULL"
    
df['wyckoff'] = df['final_structure'].progress_apply(newFunc)
df['dynamic_stability_phonons'] = stability
df = df.drop(['final_structure'], axis=1)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [12]:
print(df)
df = df[df['wyckoff'] != "NULL"]
print(df)

           material_id           composition  \
0     ENUMERATION50000          H39C41BrN2O3   
1     ENUMERATION50001             H30C17NO5   
2     ENUMERATION50002               H32C28O   
3     ENUMERATION50003        SiH36C32SBrNO6   
4     ENUMERATION50004              H26C18O5   
...                ...                   ...   
9995  ENUMERATION59995  H138.24C118.56O41.44   
9996  ENUMERATION59996             H19C24N3O   
9997  ENUMERATION59997               H9C9N3O   
9998  ENUMERATION59998            H11C23N3O2   
9999  ENUMERATION59999             H22C29NO2   

                                                wyckoff  \
0        AB41C39D2E3_mP172_4_a_41a_39a_2a_3a:Br-C-H-N-O   
1               A17B30CD5_mP212_13_17g_30g_g_5g:C-H-N-O   
2                       A28B32C_mP122_4_28a_32a_a:C-H-O   
3     AB32C36DE6FG_mP156_4_a_32a_36a_a_6a_a_a:Br-C-H...   
4                    A18B26C5_mP196_14_18e_26e_5e:C-H-O   
...                                                 ...   
9995      

In [13]:
df.to_csv('C:\\Users\\dylan\\Desktop\\generatedData\\evenMoreMorePhonons630.csv', index=False)